# Notícias G1 - 1. Extração de Dados do Portal G1

Faremos uma extração de dados que são notícias sobre diversos assuntos a partir do portal do G1 utilizando técnicas de crawling com requests e BeutifulSoup.

## Imports

Importes de bibliotecas usadas com frequência no código.

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import nltk
import numpy as np
import pandas as pd

## Método para os requests

Método que realiza um HTTP request para uma URL fornececida e retorna o resultado dessa operação contendo os dados da resposta.

In [2]:
def fazer_request_http (id_request, url):

  # Fazendo requisição HTTP:
  response = requests.get(url)

  # Testando se a conexão funcionou:
  if response.status_code != 200:
    response.raise_for_status()
  else:
    print("Status Code página {}: {}".format(id_request, response.status_code))
    response.encoding = 'utf-8' # Ajustando enconding

  return response

## Recuperando as páginas 

Realiza uma quantidade N de HTTP requests à página do portal G1, acrescentando o número da página solicitada a cada iteração, a fim de obter os dados de cada página. Cada página contém uma certa quantidade de notícias, em torno de 10 por página.

In [3]:
paginas = []

for x in range(200):
  response = fazer_request_http(x, "https://g1.globo.com/index/feed/pagina-{}.ghtml".format(x))
  html = response.text
  paginas.append(html)
  time.sleep(1)

Status Code página 0: 200
Status Code página 1: 200
Status Code página 2: 200
Status Code página 3: 200
Status Code página 4: 200
Status Code página 5: 200
Status Code página 6: 200
Status Code página 7: 200
Status Code página 8: 200
Status Code página 9: 200
Status Code página 10: 200
Status Code página 11: 200
Status Code página 12: 200
Status Code página 13: 200
Status Code página 14: 200
Status Code página 15: 200
Status Code página 16: 200
Status Code página 17: 200
Status Code página 18: 200
Status Code página 19: 200
Status Code página 20: 200
Status Code página 21: 200
Status Code página 22: 200
Status Code página 23: 200
Status Code página 24: 200
Status Code página 25: 200
Status Code página 26: 200
Status Code página 27: 200
Status Code página 28: 200
Status Code página 29: 200
Status Code página 30: 200
Status Code página 31: 200
Status Code página 32: 200
Status Code página 33: 200
Status Code página 34: 200
Status Code página 35: 200
Status Code página 36: 200
Status Code

In [4]:
# Grava os dados das páginas recuperadas:
df_paginas = pd.DataFrame(paginas, columns = ['pagina'])
df_paginas.to_csv('noticias_g1_paginas.csv', index=False)

## Capturando os links:

Faz um loop para processar cada página obtida do G1, separando as URLs de todas as notícias contidas nessas páginas. Ao final teremos um Dataframe com as URLs de todas as notícias.

In [5]:
noticias = pd.DataFrame(columns=['titulo', 'url', 'texto', 'categoria'])

for pagina in df_paginas['pagina']:
  soup = BeautifulSoup(pagina, "lxml")
  for noticia in soup.find_all('a', class_='feed-post-link'):
    if noticia.get('href') is not None and "http" in noticia.get('href'):
      href = noticia.get('href')
      pos = href.index('.com/') + 5
      categoria = href[pos:href.index("/", pos)]
      noticias = noticias.append({'titulo': noticia.text, 'url': noticia.get('href'), 'categoria': categoria}, ignore_index=True)

print("Quantidade de links: ", len(noticias))
noticias

Quantidade de links:  1982


,titulo,url,texto,categoria
0,Diretora de empresa que pretendia vender a Cov...,https://g1.globo.com/politica/cpi-da-covid/not...,NaN,politica
1,"'Estou exausta', diz Emanuela Medrades à CPI; ...",https://g1.globo.com/politica/video/video-eman...,NaN,politica
2,Grupo que intermediou vacinas procurou Bolsona...,https://g1.globo.com/politica/noticia/2021/07/...,NaN,politica
3,Média móvel de mortes por Covid no Brasil é a ...,https://g1.globo.com/bemestar/coronavirus/noti...,NaN,bemestar
4,Gilmar Mendes manda governo avaliar vacinação ...,https://g1.globo.com/politica/noticia/2021/07/...,NaN,politica
...,...,...,...,...
1977,Ministério da Agricultura registra 2 princípio...,https://g1.globo.com/economia/agronegocios/not...,NaN,economia
1978,Psiquiatra é condenado a mais de 20 anos de pr...,https://g1.globo.com/rj/rio-de-janeiro/noticia...,NaN,rj
1979,Alpinista escala Monte Everest com a mãe de 61...,https://g1.globo.com/olha-que-legal/noticia/20...,NaN,olha-que-legal
1980,PM acompanha reintegração de posse em condomín...,https://g1.globo.com/sp/sao-paulo/noticia/2021...,NaN,sp


## Recuperando as notícias:

Faz um loop realizando um HTTP request para cada notícia encontrada em cada página. Ao final, teremos um Dataframe com os dados de todas as notícias das páginas capturadas no G1.

In [8]:
for x in range(1639, noticias.index.size):
  
  noticia = noticias.loc[x]
  request = fazer_request_http(x, noticia['url'])
  soup = BeautifulSoup(request.text, "lxml")
  
  texto_noticia = ""
  for conteudo in soup.find_all('p', class_='content-text__container'):
    texto_noticia += conteudo.text
  noticia.texto = texto_noticia

  time.sleep(1)

Status Code página 1639: 200
Status Code página 1640: 200
Status Code página 1641: 200
Status Code página 1642: 200
Status Code página 1643: 200
Status Code página 1644: 200
Status Code página 1645: 200
Status Code página 1646: 200
Status Code página 1647: 200
Status Code página 1648: 200
Status Code página 1649: 200
Status Code página 1650: 200
Status Code página 1651: 200
Status Code página 1652: 200
Status Code página 1653: 200
Status Code página 1654: 200
Status Code página 1655: 200
Status Code página 1656: 200
Status Code página 1657: 200
Status Code página 1658: 200
Status Code página 1659: 200
Status Code página 1660: 200
Status Code página 1661: 200
Status Code página 1662: 200
Status Code página 1663: 200
Status Code página 1664: 200
Status Code página 1665: 200
Status Code página 1666: 200
Status Code página 1667: 200
Status Code página 1668: 200
Status Code página 1669: 200
Status Code página 1670: 200
Status Code página 1671: 200
Status Code página 1672: 200
Status Code pá

Status Code página 1922: 200
Status Code página 1923: 200
Status Code página 1924: 200
Status Code página 1925: 200
Status Code página 1926: 200
Status Code página 1927: 200
Status Code página 1928: 200
Status Code página 1929: 200
Status Code página 1930: 200
Status Code página 1931: 200
Status Code página 1932: 200
Status Code página 1933: 200
Status Code página 1934: 200
Status Code página 1935: 200
Status Code página 1936: 200
Status Code página 1937: 200
Status Code página 1938: 200
Status Code página 1939: 200
Status Code página 1940: 200
Status Code página 1941: 200
Status Code página 1942: 200
Status Code página 1943: 200
Status Code página 1944: 200
Status Code página 1945: 200
Status Code página 1946: 200
Status Code página 1947: 200
Status Code página 1948: 200
Status Code página 1949: 200
Status Code página 1950: 200
Status Code página 1951: 200
Status Code página 1952: 200
Status Code página 1953: 200
Status Code página 1954: 200
Status Code página 1955: 200
Status Code pá

In [10]:
noticias

,titulo,url,texto,categoria
0,Diretora de empresa que pretendia vender a Cov...,https://g1.globo.com/politica/cpi-da-covid/not...,"O presidente da CPI da Covid, senador Omar Az...",politica
1,"'Estou exausta', diz Emanuela Medrades à CPI; ...",https://g1.globo.com/politica/video/video-eman...,,politica
2,Grupo que intermediou vacinas procurou Bolsona...,https://g1.globo.com/politica/noticia/2021/07/...,Mensagens no celular do policial que denuncio...,politica
3,Média móvel de mortes por Covid no Brasil é a ...,https://g1.globo.com/bemestar/coronavirus/noti...,O Brasil registrou 1.613 mortes por Covid-19 ...,bemestar
4,Gilmar Mendes manda governo avaliar vacinação ...,https://g1.globo.com/politica/noticia/2021/07/...,"O ministro Gilmar Mendes, do Supremo Tribunal...",politica
...,...,...,...,...
1977,Ministério da Agricultura registra 2 princípio...,https://g1.globo.com/economia/agronegocios/not...,O Ministério da Agricultura liberou mais 29 a...,economia
1978,Psiquiatra é condenado a mais de 20 anos de pr...,https://g1.globo.com/rj/rio-de-janeiro/noticia...,Um médico psiquiatra foi condenado a mais de ...,rj
1979,Alpinista escala Monte Everest com a mãe de 61...,https://g1.globo.com/olha-que-legal/noticia/20...,Jess Wedel foi diagnosticada com câncer no ov...,olha-que-legal
1980,PM acompanha reintegração de posse em condomín...,https://g1.globo.com/sp/sao-paulo/noticia/2021...,A Polícia Militar acompanha nesta terça-feira...,sp


### Gravando Dados:

Gravando os dados em CSV para posterior utilização.

In [13]:
noticias_old = pd.read_csv('noticias_g1_baixadas.csv')
noticias = noticias_old.append(noticias)
noticias.to_csv('noticias_g1_baixadas.csv', index=False)

In [14]:
noticias

,titulo,url,texto,categoria
0,Entenda como uma ampla coalizão tirou Netanyah...,https://g1.globo.com/mundo/noticia/2021/06/13/...,O Parlamento de Israel confirmou neste doming...,mundo
1,"Milionário de direita: quem é Naftali Bennett,...",https://g1.globo.com/mundo/noticia/2021/06/13/...,"Milionário do setor de tecnologia, Naftali Be...",mundo
2,Governo de SP antecipa datas de vacinação; vej...,https://g1.globo.com/sp/sao-paulo/noticia/2021...,O governo de São Paulo antecipou em 30 dias o...,sp
3,ASSISTA: Gilberto Gil faz live de arraial no G...,https://globoplay.globo.com/arraial-do-gil/ao-...,NaN,arraial-do-gil
4,Athletico-PR bate o Grêmio e assume a liderança,https://ge.globo.com/rs/futebol/brasileirao-se...,NaN,rs
...,...,...,...,...
1977,Ministério da Agricultura registra 2 princípio...,https://g1.globo.com/economia/agronegocios/not...,O Ministério da Agricultura liberou mais 29 a...,economia
1978,Psiquiatra é condenado a mais de 20 anos de pr...,https://g1.globo.com/rj/rio-de-janeiro/noticia...,Um médico psiquiatra foi condenado a mais de ...,rj
1979,Alpinista escala Monte Everest com a mãe de 61...,https://g1.globo.com/olha-que-legal/noticia/20...,Jess Wedel foi diagnosticada com câncer no ov...,olha-que-legal
1980,PM acompanha reintegração de posse em condomín...,https://g1.globo.com/sp/sao-paulo/noticia/2021...,A Polícia Militar acompanha nesta terça-feira...,sp
